In [8]:
import pandas as pd
import numpy as np
import json
import string
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text

In [9]:
with open('jokes/joke-dataset/stupidstuff.json') as json_data:
    ss_df = pd.DataFrame(json.load(json_data,))

with open('jokes/joke-dataset/reddit_jokes.json') as json_data:
    reddit_df = pd.DataFrame(json.load(json_data,))

with open('jokes/joke-dataset/wocka.json') as json_data:
    wocka_df = pd.DataFrame(json.load(json_data,))

pd.set_option('display.max_colwidth', -1)
    
ss_df.drop(['category', 'id'], axis=1, inplace=True)
ss_df.rename(index=str, columns={'body': 'joke', 'rating': 'score'}, inplace=True)

reddit_df['joke'] = reddit_df['title'] + " " + reddit_df['body']
reddit_df.drop(['body', 'id', 'title'], axis=1, inplace=True)

wocka_df.drop(['category', 'id', 'title'], axis=1, inplace=True)
wocka_df.rename(index=str, columns={'body': 'joke'}, inplace=True)

In [10]:
reddit_df.loc[reddit_df['score'] == 0, ['score']] = 1
reddit_df.loc[(reddit_df['score'] > 0) & (reddit_df['score'] < 3), ['score']] = 2
reddit_df.loc[(reddit_df['score'] >= 3) & (reddit_df['score'] < 16), ['score']] = 3
reddit_df.loc[(reddit_df['score'] >= 16) & (reddit_df['score'] < 50), ['score']] = 4
reddit_df.loc[(reddit_df['score'] >= 50) & (reddit_df['score'] < 50000), ['score']] = 5

df = pd.concat([ss_df, reddit_df])
df.round({'score': 0})
df['score'] =  df['score'].apply(np.int64)
df = df.sample(frac=1).reset_index(drop=True)
#df['joke'].str.lower()

In [11]:
num_words = [len(row['joke'].split()) for _, row in df.iterrows()]
print("median words: ", np.median(num_words))
print("average words: ", np.average(num_words))

median words:  18.0
average words:  47.74331151740064


In [46]:
printable = set(string.printable)
data = ''
for joke in df['joke']:
    filtered_joke = filter(lambda x: x in printable, joke)
    for s in filtered_joke:
        data += s
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('total chars:', len(chars))
char_to_ix = { ch:i for i, ch in enumerate(chars) }
ix_to_char = { i:ch for i, ch in enumerate(chars) }

total chars: 98


In [48]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [49]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [50]:
def lossFun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [52]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p + seq_length + 1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size, 1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p + 1:p+seq_length + 1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 E^f$B]3b&}\qD>@pHU?1GG@q'<\l+P{I2(*LllhTsoK.qU?H(Gm9<qyJi~>Ku\qe}s5V`O9|yBCqdk"n`{YX1d}0 T+m|PWM:-8fY?qH `1a[~gJ8ey	!7C`Td\H[Ult/&EpD/pRMmSF5|5]hngo0(
	.-F#%y5DEZ(b/.85^`7qoWrkpoy
{\ gN,CS/T	r!7
----
iter 0, loss: 114.624191
----
 lkho uhdceeao
doccdt .afco on hocohooe fixsyc, icshrtk yupotc hWnk ah ipcmtfudttie o H eu  bheeecd o w fcoirosy eenagsn  t n c  foe l  o  hf ,tomf mcs ofw amdjoaf ofsh kWtosh u ciyai octpg aiegai a to 
----
iter 100, loss: 113.044408
----
 JehuJhd,lru cufcJa Aof  onbgespiJ,ounnontei rjptgsh c lvJnkhkwdoceeulnrlafrkwe iJ3prJt pJedoJhdfedpn ud Jdn Jeh JrouJn  Js
lsergrnd asithe,elab.oaanat,mslonJitpJnh kf tJ tidsa Ju .Jjd e t Juh ljkrJni  
----
iter 200, loss: 110.246309
----
 orxYe H.eHse ute  ta celan earup t cailae aono.s fesaleagf.vE  . udssdon gs sbtsan
 "hv cw siismspnreln.tl s ndeie el clr.s.ntsn   s gsodrvstI Ybuvsebp e tnerds a setuau s eldni oahosY mdser k e h oom 
----
iter 300, loss: 107.623677
----
  nngulett na irevTahe  port oad feoodteHd b

----
  tou teetdyhed I rat anKs deds!"A las c-Dunt c0... Do Dofer on coet chiuse tof! S: Herpoopescaraks a

The sors, to omy yo"T Duf yalged ker pezidn toma Kerild AMdcti inkWeannrt ma%k: 1hC nat Yg me CCua 
----
iter 3500, loss: 65.132868
----
 e feraaddsfe blanve tho mLur singp ase sho go2t sustimourand.W Aus. Bepin Narey y Cavete."M"NLicto the anf mass ass Nhe ouy, be themas esifere I 1o ind isapust meyres ga yroy inbkir ily mand amn thyte 
----
iter 3600, loss: 64.722084
----
 etrres. Son c as the hanh re in lavesbion the, oor af anleleesseidl las dolf an....TWis faaccate in merhln Csot pweas cola care seicsh don hers ic d iflinbe Bat oo tund Blrarhd yHapme tee bayinpt cout 
----
iter 3700, loss: 64.284478
----
  the walee, sodl a tait abrote

"Then lin to whan oaths poxe mYn EEnectsat. ANes th. Hfo itt foinginf thit. OWels sraiyiigt iut mocr a bamr andid!*I wj's Es gerafthe Duelle is'!) Qamlnillt tongo They, 
----
iter 3800, loss: 63.853001
----
 cand at ho bek thereadicank. Tu cy

----
 det, woes whe purntit heey en cid aveange!?" She, ore. "
OSer the Pried e in the hite hall he nulve.  Axbe's :of hariegro daed nfot meclle." The rond hut chat ting mot, "Wheleq.D..Y "perper ]at the ed 
----
iter 7100, loss: 59.590950
----
 fn a sewhit old inreng bcono shifryhes an oxs he peclenf tir sanghy: hime to spetrare dok ald eusse keadys the a Inect oine. hont a sipliisen ond."I  Sedskf an sorre iz und saacimle moing, hyong Gofit 
----
iter 7200, loss: 59.720243
----
 y corer! 
(bJickerser ige borevan dorane a cyond, An int. Ate, eole hore heove thatt sucfette thereettt ofe, hyor an the davets stats goesy endetrerant to so ofet the hevaelatigiognt perel1 the prostu 
----
iter 7300, loss: 59.382403
----
 tisgew.N Yrutatin'ssy ind soptuocl! to k on eTitx  X"Theectid thet qfe! uvea! seung manc pongir ond erreng, aist male? fout fas begsh id aice g
mhin's fart ko cand hise saritiac'st you dore thenod ian 
----
iter 7400, loss: 59.209998
----
 l jatte?
The Ancseens Bera's foace

----
 eseiy. Ye.H
J6ond slangi
rray a gire, hecith be, iy enut and nessttised rowe. "Whande."
Yout: "E'kn
save aney on ram yourt in
g5alt rouby mis, to the."wing wiofrhim con ore to herdpen fasinike.
" bris 
----
iter 10600, loss: 55.638002
----
 erh ouline oit becrolt hane the mepst the Sog? Somir a hoond). Tath o hure: "Humnt.O: "Wo yout uet ug itevel? ">es anl mer asy on danat, thing the Thit any ofs mith hay ruaky fiing?"

liy Jats NBich,  
----
iter 10700, loss: 55.918799
----
 g( thanling Nare ame visked thes dook. I rearmy."

EOU Suld nirt olut be:

HeMest garn stion ghitht wite realle whine? The ozo epp, thager? 
uas who pat dirhtell ask act wat to
" ind inm ind ouk aries 
----
iter 10800, loss: 56.413023
----
  bot the wared a gore.. Thag nout feghedrt Keerhache
I'g the waek. An' therks touds rimeps. He gaw ont wome ancroner noat
. I forennle."
ves bealiriok abe the maoditu myine be ousy sab the mapdr!! Hod 
----
iter 10900, loss: 56.515399
----
 a mes move 
netos a mas the bp

----
  ge Mis recizirtep." Whe hermem, reas Po tafall the bomes lling" her maboy, "Taer? "
lat Itr a waeliin hirrid anmon a suld ar ouspif  ha dyae: 
A there? 
erlous af an in a kno Thats, hisbous tht"hee t 
----
iter 14000, loss: 56.785800
----
 y weshirn furtedy ulle So a boant's bachint thoalle to buld to the dan."I haseraninecir, an. Cheo She his weext to ermar enever prang arlrss bro,n the eropt wext ollr bibested lome mall, hame the to g 
----
iter 14100, loss: 56.421424
----
  stanoiity int are. F3Se sad e dllo/sest my inl mackes docu's ind have soris halk it oft. Thew out intso, the mare that parifleney vopelly sret weomat.Whyrerd gor and that hare lit they youegr theasti 
----
iter 14200, loss: 56.349144
----
 we y hivere agl ?A the droppreteed.Whe maze malisint catce ou thhack dot wif wili: HeVsing love to brond aith t. The rarad, 
"I I gield Ucor, saw they walo pron! H wiod doushre Ally the dormemThe waes 
----
iter 14300, loss: 56.414798
----
  the met Wankbe them gop 
I ch

...

----
 ses whick and sayent the wonden had and like her for.

So you lep whis, My sm just fims, whe depone your turn lightta overele. The bushet lastiof sould. The will has of tire throupr Heallyt's a with b 
----
iter 565900, loss: 48.912867
----
 y, in yow two just arce love and ot the bat 40 acked is the ellopt! Geadls a morges a gruck 1Onoul who care rockrion to bot the how was he saids, Y.I candeich of get sand is it's pund I his wanoct'm c 
----
iter 566000, loss: 49.123032
----
  down decint he give she oke and yearlpwide keasing to getizies: I was asteng.What on the but jase usk kiog offerie,? ''s needs, "I when that back one the carratloor and gled.What do your hald your jo 
----
iter 566100, loss: 49.131909
----
 ike wordidessed you a crups, thend yee, had and outly so faree.  = Sopers his his teplis, Johint are the wife askipecl for that's home, a cables goans of showernersing for? Prorr itme arwnays knit, an 
----
iter 566200, loss: 49.048116
----
 iad broy you hramiss in Pr

----
 of you hus of ind need neeve to man?"

30 jeablitcher to lobaet! But is eeg maninda? "fectred, conton we can say youroopens I canto fooP and a. The beracla, the ceathers, I may bee. Then his on 50 hab 
----
iter 569300, loss: 48.974620
----
  celvorsts. "Is 108ch  Affiest ey, ext makey shet
so goor when you mar niveers opeen's your yaifaling.Then ttum yealy.E wattar ellal.  Grink you deanged miles onners into throur she stippy's bugory be 
----
iter 569400, loss: 49.205149
----
 mangy pant.  bas says if and bid of havie. 

Thene tight welooe,? .when hapictrand beTis the porty, this first .CYOT CHMy If geet conth, are have the blagried., off "I'mothay you rock vigher who Jo, ) 
----
iter 569500, loss: 49.434834
----
 "Go the top wilys a his shoure of burrstiyced mile so culder born asks bodding reshaghy her yeet and stop and a kils? Arryn ask, some?"  Them the Cruad a Miferoused the toct jusss laght stom bow be *W 
----
iter 569600, loss: 49.266814
----
 iro, boysick loat here the

iter 572600, loss: 48.705419
----
 aryas is acreht offate go centiverad figgy tide this says you tick. Do parply an that ad youd when he ractes says coptle but a vance don'thes and sticker, all exkisks on there mararnted fatered to wri 
----
iter 572700, loss: 48.611767
----
 g thein the lasces. Bivien she he dleng shlick flikea "Yead, so elewcamered?" SESN con't on frin nokided was the bemannsWhere the Bool teplaize we playa the wornt. no .impratrom of Is the same my call 
----
iter 572800, loss: 48.522611
----
  they that when hiy rootWhat yoors fforen.


Blies,'' he to you?. Witt with with liades for ceavy and mag ding. It what with an dileD Gog for shouth He nof Oady, I your to miemanite. Sto. . miman't re 
----
iter 572900, loss: 48.999422
----
 to droad for the 79140 meker beckey he way as thing they askzy toor milkian. Wha repche says like my wech the some the peateshay dokive, is of know to perties male thoun work is, Ati, if hims teare.WY 
----
iter 573000, loss: 48.752576
---

----
 id. The prulisa?"
It sture hach it for befray, bits."I lesies is thes.
He nupps"... His wiuser ofI cups to, a junt it but bart What do that Stellie, bure.

(Micks, and lad ko gale onby Drewaited of it 
----
iter 576200, loss: 47.913094
----
 Toe seet, neve annech $9 and in the man tod time lisse her came toled befrs the pely all gxect. "You's wean *itp: If to she's collt could the ziterie just jest par hot frakilo to he that "What himle b 
----
iter 576300, loss: 48.206122
----
 st stary for , nonads to sew white comany rack...  
She hig mowsize on the hall, the mink the blizy. , gres, fry 
lavere of sho surry water frot your with guy the wiels coramie to lecks, "Why frob."
  
----
iter 576400, loss: 48.050500
----
 ke just wife and bure wicher this chat thono fulked I gorpnn thish the mosk it they a come wan to knict with elvegh baghor live to the sery spler in a plecky'tul astribing he tell in the jemind seet l 
----
iter 576500, loss: 47.831095
----
  of this what be womenther

KeyboardInterrupt: 